In [1]:
import pandas as pd
import requests
import re
import scrapy

from bs4 import BeautifulSoup
from scrapy.crawler import CrawlerProcess

In [2]:
pbp = [
{"away_starting_pitcher": "Grant Suponchick", "home_starting_pitcher": "Christie, Hank", "away_team": "OMAHA", "home_team": "Northwestern"},
{"half_inning": "OMAHA - Top of 1st", "offense_team": "OMAHA", "plays": ["Thibodeau,C singled to center field.", "Thibodeau,C stole second.", "Hughes,H struck out looking.", "Krumwiede,K grounded out to ss.", "Rogers,B grounded out to p."]},
{"half_inning": "Northwestern - Bottom of 1st", "offense_team": "Northwestern", "plays": ["Dickey grounded out to p.", "Dunn, J. singled to third base.", "Erro singled to right center; Dunn, J. advanced to third.", "Erro stole second; Dunn, J. stole home.", "Claeys struck out swinging, out at first c to 1b; Erro advanced to third.", "Lind struck out looking."]},
{"half_inning": "OMAHA - Top of 2nd", "offense_team": "OMAHA", "plays": ["Palensky,B struck out swinging.", "Caniglia,A flied out to rf.", "Smejkal,P flied out to rf."]},
{"half_inning": "Northwestern - Bottom of 2nd", "offense_team": "Northwestern", "plays": ["Dunn, D. grounded out to 3b.", "Maxwell flied out to cf.", "Kelly grounded out to ss."]},
{"half_inning": "OMAHA - Top of 3rd", "offense_team": "OMAHA", "plays": ["Daugherty,T singled up the middle.", "Gamm,M grounded out to p, bunt; Daugherty,T advanced to second.", "Thibodeau,C singled up the middle, RBI; Daugherty,T scored.", "Hughes,H singled through the right side; Thibodeau,C advanced to second.", "Hughes,H advanced to second on a passed ball; Thibodeau,C advanced to third on a passed ball.", "Krumwiede,K tripled to right center, 2 RBI; Hughes,H scored; Thibodeau,C scored.", "Rogers,B grounded out to 1b unassisted.", "Palensky,B doubled to center field, RBI; Krumwiede,K scored.", "Caniglia,A lined out to cf."]},
{"half_inning": "Northwestern - Bottom of 3rd", "offense_team": "Northwestern", "plays": ["Bourbon tripled to right center.", "Dickey grounded out to ss, RBI; Bourbon scored.", "Dunn, J. singled to right field.", "Erro hit by pitch; Dunn, J. advanced to second.", "Claeys reached on a fielder's choice; Erro advanced to second; Dunn, J. out at third ss to 3b.", "Lind walked; Claeys advanced to second; Erro advanced to third.", "Dunn, D. flied out to rf."]},
{"half_inning": "OMAHA - Top of 4th", "offense_team": "OMAHA", "plays": ["Smejkal,P singled to third base.", "Daugherty,T hit by pitch; Smejkal,P advanced to second.", "Gamm,M reached on a fielder's choice; Daugherty,T out at second 3b to 2b; Smejkal,P advanced to third.", "Bordignon to p for Christie.", "Thibodeau,C singled to left field, advanced to second on the throw, RBI; Gamm,M advanced to third; Smejkal,P scored.", "Hughes,H reached on an error by ss; Thibodeau,C scored, unearned; Gamm,M scored.", "Krumwiede,K grounded out to 3b; Hughes,H advanced to second.", "Rogers,B tripled down the rf line, RBI; Hughes,H scored, unearned.", "Palensky,B struck out looking."]},
{"half_inning": "Northwestern - Bottom of 4th", "offense_team": "Northwestern", "plays": ["Maxwell flied out to lf.", "Kelly popped up to ss.", "Bourbon grounded out to 2b."]},
{"half_inning": "OMAHA - Top of 5th", "offense_team": "OMAHA", "plays": ["Caniglia,A grounded out to ss.", "Smejkal,P singled through the left side.", "Daugherty,T struck out swinging.", "Gamm,M reached on a fielder's choice; Smejkal,P out at second 2b to ss."]},
{"half_inning": "Northwestern - Bottom of 5th", "offense_team": "Northwestern", "plays": ["Dickey singled to center field.", "Dunn, J. flied out to rf.", "Erro flied out to cf.", "Dickey out at second c to ss, caught stealing."]},
{"half_inning": "OMAHA - Top of 6th", "offense_team": "OMAHA", "plays": ["Thibodeau,C walked.", "Hughes,H grounded into double play 2b to ss to 1b; Thibodeau,C out on the play.", "Krumwiede,K singled to center field.", "Krumwiede,K advanced to second on a wild pitch.", "Rogers,B walked.", "Palensky,B struck out looking."]},
{"half_inning": "Northwestern - Bottom of 6th", "offense_team": "Northwestern", "plays": ["Claeys singled to shortstop.", "Lind grounded out to 2b; Claeys advanced to second.", "Dunn, D. singled to shortstop; Claeys advanced to third.", "Maxwell hit by pitch; Dunn, D. advanced to second.", "Moore,M to p for Suponchick,G.", "Kelly walked, RBI; Maxwell advanced to second; Dunn, D. advanced to third; Claeys scored.", "Bourbon grounded out to ss, RBI; Kelly advanced to second; Maxwell advanced to third; Dunn, D. scored.", "Dickey struck out swinging."]},
{"half_inning": "OMAHA - Top of 7th", "offense_team": "OMAHA", "plays": ["Gannon to p for Bordignon.", "Caniglia,A walked.", "Smejkal,P singled to right field; Caniglia,A advanced to second.", "Daugherty,T singled to third base, bunt; Smejkal,P advanced to second; Caniglia,A advanced to third.", "Gamm,M singled through the left side, RBI; Daugherty,T advanced to second; Smejkal,P advanced to third; Caniglia,A scored.", "Thibodeau,C singled to center field, RBI; Gamm,M advanced to second; Daugherty,T advanced to third; Smejkal,P scored.", "Lass to p for Gannon.", "Hughes,H walked, RBI; Thibodeau,C advanced to second; Gamm,M advanced to third; Daugherty,T scored.", "Krumwiede,K reached on a fielder's choice, RBI; Hughes,H out at second 3b to 2b; Thibodeau,C advanced to third; Gamm,M scored.", "Rogers,B walked; Krumwiede,K advanced to second.", "Palensky,B singled to center field, RBI; Rogers,B advanced to second; Krumwiede,K advanced to third; Thibodeau,C scored.", "Caniglia,A singled to left field, RBI; Palensky,B advanced to second; Rogers,B advanced to third; Krumwiede,K scored.", "Smejkal,P popped up to rf.", "Daugherty,T flied out to cf."]},
{"half_inning": "Northwestern - Bottom of 7th", "offense_team": "Northwestern", "plays": ["Dunn, J. struck out looking.", "Erro walked.", "Claeys struck out looking.", "Erro stole second.", "Erro stole third.", "Lind struck out swinging."]},
{"half_inning": "OMAHA - Top of 8th", "offense_team": "OMAHA", "plays": ["No play.", "O'Laughlin to lf for Maxwell.", "No play.", "Gamm,M lined out to cf.", "Thibodeau,C walked.", "Hughes,H grounded into double play p to ss to 1b; Thibodeau,C out on the play."]},
{"half_inning": "Northwestern - Bottom of 8th", "offense_team": "Northwestern", "plays": ["Dunn, D. tripled to right center.", "O'Laughlin singled through the right side, RBI; Dunn, D. scored.", "Kelly lined out to rf.", "Bourbon struck out swinging.", "Dickey doubled down the rf line; O'Laughlin advanced to third.", "Dunn, J. singled to left field, 2 RBI; Dickey scored; O'Laughlin scored.", "Dunn, J. stole second.", "Erro reached on a throwing error by ss; Dunn, J. advanced to third.", "Jones to c for Claeys.", "Dunn, J. scored on the throw, unearned.", "Jones walked; Erro advanced to second.", "Lind struck out swinging."]},
{"half_inning": "OMAHA - Top of 9th", "offense_team": "OMAHA", "plays": ["Krumwiede,K singled to center field.", "Rogers,B grounded into double play 2b to ss to 1b; Krumwiede,K out on the play.", "Palensky,B flied out to rf."]},
{"half_inning": "Northwestern - Bottom of 9th", "offense_team": "Northwestern", "plays": ["Steier,T to p for Moore,M.", "Dunn, D. flied out to cf.", "O'Laughlin grounded out to ss.", "Kelly grounded out to 2b."]}
]

d = []
# pbp = list(result)
info = pbp[0]
# info['away_team'] = team_ids[info['away_team']]
# info['home_team'] = team_ids[info['home_team']]
pitcher_against = {
    info['away_team']: info['home_starting_pitcher'],
    info['home_team']: info['away_starting_pitcher']
}

for half_inning in pbp[1:]:
    d.extend([{
        'half_inning': half_inning['half_inning'],
        'play': p,
        'offense_team': half_inning['offense_team']
    } for p in half_inning['plays']])

# REMOVE 'NO PLAY'
d = list(filter(lambda a: a['play'] != 'No play.', d))

# Extract pa_result and batted_ball_location
for play in d:
    
    play['pitcher'] = pitcher_against[play['offense_team']]
    
    # get name
    pattern = r"(\w+'?\w+\,[A-Z]|\w+'?\w+,\s[A-Z].|\w+'?\w+)"
    match = re.match(pattern, play['play'])
    if bool(match):
        play['name'] = match.group(0)
    else:
        print('No name')

    # get pitcher substitution
    pattern = r"to p for"
    match = re.search(pattern, play['play'])
    if bool(match):
        pitcher_against[play['offense_team']] = play['name']

    # if possible, get result
    # stolen base
    pattern = r"(stole)"
    match = re.search(pattern, play['play'])
    if bool(match):
        pa_result = match.group(0)

    # struck out looking, struck out swinging, walked, hit by pitch
    pattern = r"(struck out looking|struck out swinging|walked|hit by pitch)"
    match = re.search(pattern, play['play'])

    if bool(match):
        pa_result = match.group(0)

    # ground, lined, flied ...
    pattern = r"(grounded|lined|flied|popped|reached|singled|doubled|tripled|homered).*(p|1b|2b|3b|ss|lf|cf|rf|left field|center field|right field|first base|second base|third base|left center|right center|left side|middle|right side)\b"
    match = re.search(pattern, play['play'])

    if bool(match):
        play['pa_result'] = match.group(1)
        play['batted_ball_location'] = match.group(2)

    # double play
    pattern = r"(grounded|lined|flied|popped|reached|singled|doubled|tripled|homered).*(?<!to )(p|1b|2b|3b|ss|lf|cf|rf)\b"
    match = re.search(pattern, play['play'])

    if bool(match):
        play['batted_ball_location'] = match.group(2)
    
pd.DataFrame(d)

,batted_ball_location,half_inning,name,offense_team,pa_result,pitcher,play
0,center field,OMAHA - Top of 1st,"Thibodeau,C",OMAHA,singled,"Christie, Hank","Thibodeau,C singled to center field."
1,NaN,OMAHA - Top of 1st,"Thibodeau,C",OMAHA,NaN,"Christie, Hank","Thibodeau,C stole second."
2,NaN,OMAHA - Top of 1st,"Hughes,H",OMAHA,NaN,"Christie, Hank","Hughes,H struck out looking."
3,ss,OMAHA - Top of 1st,"Krumwiede,K",OMAHA,grounded,"Christie, Hank","Krumwiede,K grounded out to ss."
4,p,OMAHA - Top of 1st,"Rogers,B",OMAHA,grounded,"Christie, Hank","Rogers,B grounded out to p."
5,p,Northwestern - Bottom of 1st,Dickey,Northwestern,grounded,Grant Suponchick,Dickey grounded out to p.
6,third base,Northwestern - Bottom of 1st,"Dunn, J.",Northwestern,singled,Grant Suponchick,"Dunn, J. singled to third base."
7,right center,Northwestern - Bottom of 1st,Erro,Northwestern,singled,Grant Suponchick,"Erro singled to right center; Dunn, J. advance..."
8,NaN,Northwestern - Bottom of 1st,Erro,Northwestern,NaN,Grant Suponchick,"Erro stole second; Dunn, J. stole home."
9,NaN,Northwestern - Bottom of 1st,Claeys,Northwestern,NaN,Grant Suponchick,"Claeys struck out swinging, out at first c to ..."


In [60]:
YEAR = 2018
teams = {
    20011: 'Minnesota',
    20417: 'Purdue',
    20183: 'Michigan',
    20178: 'Illinois',
    20452: 'Indiana',
    20179: 'Iowa',
    20152: 'Ohio State',
    20145: 'Michigan State',
    20213: 'Maryland',
    20066: 'Nebraska',
    20097: 'Rutgers',
    20037: 'Northwestern',
    20348: 'Penn State'
}

In [70]:
df = pd.DataFrame()
for team in teams:
    r = requests.get("http://www.thebaseballcube.com/teams/stats.asp?Y=" + str(YEAR) + "&T=" + str(team))
    soup = BeautifulSoup(r.text, 'html.parser')
    data_grids = soup.select('.dataGrid')
    batting_table = data_grids[0].find_all('table')[1]
    df = df.append(pd.read_html(str(batting_table), header=0)[0], ignore_index=True)

In [82]:
df.sort_values('wOBA', ascending=False).loc()

,Unif,Player,Pos,Ba,Age,HiLvl,G,AB,R,H,...,wOBA,BB%,SO%,K-BB,HR%,AB/HR,XBH%,Draft Information,Status,Current Org (Level)
209,Batting Totals: (16 players),50,1691,301,455,66,15,18,259,82,...,93.900,21.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,Batting Totals: (16 players),50,1691,301,455,66,15,18,259,82,...,93.900,21.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,Batting Totals: (16 players),52,1741,222,426,57,13,20,190,99,...,87.100,21.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,Batting Totals: (16 players),52,1741,222,426,57,13,20,190,99,...,87.100,21.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,Batting Totals: (17 players),49,1625,222,379,79,17,21,203,47,...,77.400,30.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,Batting Totals: (17 players),49,1625,222,379,79,17,21,203,47,...,77.400,30.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,Batting Totals: (16 players),59,1919,346,538,101,7,27,309,95,...,71.100,25.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,Batting Totals: (16 players),59,1919,346,538,101,7,27,309,95,...,71.100,25.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,Batting Totals: (17 players),49,1641,248,411,71,12,28,222,90,...,58.600,27.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,Batting Totals: (20 players),54,1857,342,508,103,12,34,306,90,...,54.600,29.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
pd.DataFrame([
{"half_inning": "OMAHA - Top of 1st", "play": "Thibodeau,C singled to center field.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Thibodeau,C", "pa_result": "singled", "batted_ball_location": "center field"},
{"half_inning": "OMAHA - Top of 1st", "play": "Thibodeau,C stole second.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Thibodeau,C", "pa_result": "stole"},
{"half_inning": "OMAHA - Top of 1st", "play": "Hughes,H struck out looking.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Hughes,H", "pa_result": "struck out looking"},
{"half_inning": "OMAHA - Top of 1st", "play": "Krumwiede,K grounded out to ss.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Krumwiede,K", "pa_result": "grounded", "batted_ball_location": "ss"},
{"half_inning": "OMAHA - Top of 1st", "play": "Rogers,B grounded out to p.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Rogers,B", "pa_result": "grounded", "batted_ball_location": "p"},
{"half_inning": "Northwestern - Bottom of 1st", "play": "Dickey grounded out to p.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Dickey", "pa_result": "grounded", "batted_ball_location": "p"},
{"half_inning": "Northwestern - Bottom of 1st", "play": "Dunn, J. singled to third base.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Dunn, J.", "pa_result": "singled", "batted_ball_location": "third base"},
{"half_inning": "Northwestern - Bottom of 1st", "play": "Erro singled to right center; Dunn, J. advanced to third.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Erro", "pa_result": "singled", "batted_ball_location": "right center"},
{"half_inning": "Northwestern - Bottom of 1st", "play": "Erro stole second; Dunn, J. stole home.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Erro", "pa_result": "stole"},
{"half_inning": "Northwestern - Bottom of 1st", "play": "Claeys struck out swinging, out at first c to 1b; Erro advanced to third.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Claeys", "pa_result": "struck out swinging"},
{"half_inning": "Northwestern - Bottom of 1st", "play": "Lind struck out looking.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Lind", "pa_result": "struck out looking"},
{"half_inning": "OMAHA - Top of 2nd", "play": "Palensky,B struck out swinging.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Palensky,B", "pa_result": "struck out swinging"},
{"half_inning": "OMAHA - Top of 2nd", "play": "Caniglia,A flied out to rf.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Caniglia,A", "pa_result": "flied", "batted_ball_location": "rf"},
{"half_inning": "OMAHA - Top of 2nd", "play": "Smejkal,P flied out to rf.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Smejkal,P", "pa_result": "flied", "batted_ball_location": "rf"},
{"half_inning": "Northwestern - Bottom of 2nd", "play": "Dunn, D. grounded out to 3b.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Dunn, D.", "pa_result": "grounded", "batted_ball_location": "3b"},
{"half_inning": "Northwestern - Bottom of 2nd", "play": "Maxwell flied out to cf.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Maxwell", "pa_result": "flied", "batted_ball_location": "cf"},
{"half_inning": "Northwestern - Bottom of 2nd", "play": "Kelly grounded out to ss.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Kelly", "pa_result": "grounded", "batted_ball_location": "ss"},
{"half_inning": "OMAHA - Top of 3rd", "play": "Daugherty,T singled up the middle.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Daugherty,T", "pa_result": "singled", "batted_ball_location": "p"},
{"half_inning": "OMAHA - Top of 3rd", "play": "Gamm,M grounded out to p, bunt; Daugherty,T advanced to second.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Gamm,M", "pa_result": "grounded", "batted_ball_location": "p"},
{"half_inning": "OMAHA - Top of 3rd", "play": "Thibodeau,C singled up the middle, RBI; Daugherty,T scored.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Thibodeau,C", "pa_result": "singled", "batted_ball_location": "p"},
{"half_inning": "OMAHA - Top of 3rd", "play": "Hughes,H singled through the right side; Thibodeau,C advanced to second.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Hughes,H", "pa_result": "singled", "batted_ball_location": "right side"},
{"half_inning": "OMAHA - Top of 3rd", "play": "Hughes,H advanced to second on a passed ball; Thibodeau,C advanced to third on a passed ball.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Hughes,H", "pa_result": ""},
{"half_inning": "OMAHA - Top of 3rd", "play": "Krumwiede,K tripled to right center, 2 RBI; Hughes,H scored; Thibodeau,C scored.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Krumwiede,K", "pa_result": "tripled", "batted_ball_location": "right center"},
{"half_inning": "OMAHA - Top of 3rd", "play": "Rogers,B grounded out to 1b unassisted.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Rogers,B", "pa_result": "grounded", "batted_ball_location": "1b"},
{"half_inning": "OMAHA - Top of 3rd", "play": "Palensky,B doubled to center field, RBI; Krumwiede,K scored.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Palensky,B", "pa_result": "doubled", "batted_ball_location": "center field"},
{"half_inning": "OMAHA - Top of 3rd", "play": "Caniglia,A lined out to cf.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Caniglia,A", "pa_result": "lined", "batted_ball_location": "cf"},
{"half_inning": "Northwestern - Bottom of 3rd", "play": "Bourbon tripled to right center.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Bourbon", "pa_result": "tripled", "batted_ball_location": "right center"},
{"half_inning": "Northwestern - Bottom of 3rd", "play": "Dickey grounded out to ss, RBI; Bourbon scored.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Dickey", "pa_result": "grounded", "batted_ball_location": "ss"},
{"half_inning": "Northwestern - Bottom of 3rd", "play": "Dunn, J. singled to right field.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Dunn, J.", "pa_result": "singled", "batted_ball_location": "right field"},
{"half_inning": "Northwestern - Bottom of 3rd", "play": "Erro hit by pitch; Dunn, J. advanced to second.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Erro", "pa_result": "hit by pitch"},
{"half_inning": "Northwestern - Bottom of 3rd", "play": "Claeys reached on a fielder's choice; Erro advanced to second; Dunn, J. out at third ss to 3b.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Claeys", "pa_result": "reached", "batted_ball_location": "ss"},
{"half_inning": "Northwestern - Bottom of 3rd", "play": "Lind walked; Claeys advanced to second; Erro advanced to third.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Lind", "pa_result": "walked"},
{"half_inning": "Northwestern - Bottom of 3rd", "play": "Dunn, D. flied out to rf.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Dunn, D.", "pa_result": "flied", "batted_ball_location": "rf"},
{"half_inning": "OMAHA - Top of 4th", "play": "Smejkal,P singled to third base.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Smejkal,P", "pa_result": "singled", "batted_ball_location": "third base"},
{"half_inning": "OMAHA - Top of 4th", "play": "Daugherty,T hit by pitch; Smejkal,P advanced to second.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Daugherty,T", "pa_result": "hit by pitch"},
{"half_inning": "OMAHA - Top of 4th", "play": "Gamm,M reached on a fielder's choice; Daugherty,T out at second 3b to 2b; Smejkal,P advanced to third.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Gamm,M", "pa_result": "reached", "batted_ball_location": "3b"},
{"half_inning": "OMAHA - Top of 4th", "play": "Bordignon to p for Christie.", "offense_team": "OMAHA", "pitcher": "Christie, Hank", "name": "Bordignon", "pa_result": ""},
{"half_inning": "OMAHA - Top of 4th", "play": "Thibodeau,C singled to left field, advanced to second on the throw, RBI; Gamm,M advanced to third; Smejkal,P scored.", "offense_team": "OMAHA", "pitcher": "Bordignon", "name": "Thibodeau,C", "pa_result": "singled", "batted_ball_location": "left field"},
{"half_inning": "OMAHA - Top of 4th", "play": "Hughes,H reached on an error by ss; Thibodeau,C scored, unearned; Gamm,M scored.", "offense_team": "OMAHA", "pitcher": "Bordignon", "name": "Hughes,H", "pa_result": "reached", "batted_ball_location": "ss"},
{"half_inning": "OMAHA - Top of 4th", "play": "Krumwiede,K grounded out to 3b; Hughes,H advanced to second.", "offense_team": "OMAHA", "pitcher": "Bordignon", "name": "Krumwiede,K", "pa_result": "grounded", "batted_ball_location": "3b"},
{"half_inning": "OMAHA - Top of 4th", "play": "Rogers,B tripled down the rf line, RBI; Hughes,H scored, unearned.", "offense_team": "OMAHA", "pitcher": "Bordignon", "name": "Rogers,B", "pa_result": "tripled", "batted_ball_location": "rf"},
{"half_inning": "OMAHA - Top of 4th", "play": "Palensky,B struck out looking.", "offense_team": "OMAHA", "pitcher": "Bordignon", "name": "Palensky,B", "pa_result": "struck out looking"},
{"half_inning": "Northwestern - Bottom of 4th", "play": "Maxwell flied out to lf.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Maxwell", "pa_result": "flied", "batted_ball_location": "lf"},
{"half_inning": "Northwestern - Bottom of 4th", "play": "Kelly popped up to ss.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Kelly", "pa_result": "popped", "batted_ball_location": "p"},
{"half_inning": "Northwestern - Bottom of 4th", "play": "Bourbon grounded out to 2b.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Bourbon", "pa_result": "grounded", "batted_ball_location": "2b"},
{"half_inning": "OMAHA - Top of 5th", "play": "Caniglia,A grounded out to ss.", "offense_team": "OMAHA", "pitcher": "Bordignon", "name": "Caniglia,A", "pa_result": "grounded", "batted_ball_location": "ss"},
{"half_inning": "OMAHA - Top of 5th", "play": "Smejkal,P singled through the left side.", "offense_team": "OMAHA", "pitcher": "Bordignon", "name": "Smejkal,P", "pa_result": "singled", "batted_ball_location": "left side"},
{"half_inning": "OMAHA - Top of 5th", "play": "Daugherty,T struck out swinging.", "offense_team": "OMAHA", "pitcher": "Bordignon", "name": "Daugherty,T", "pa_result": "struck out swinging"},
{"half_inning": "OMAHA - Top of 5th", "play": "Gamm,M reached on a fielder's choice; Smejkal,P out at second 2b to ss.", "offense_team": "OMAHA", "pitcher": "Bordignon", "name": "Gamm,M", "pa_result": "reached", "batted_ball_location": "2b"},
{"half_inning": "Northwestern - Bottom of 5th", "play": "Dickey singled to center field.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Dickey", "pa_result": "singled", "batted_ball_location": "center field"},
{"half_inning": "Northwestern - Bottom of 5th", "play": "Dunn, J. flied out to rf.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Dunn, J.", "pa_result": "flied", "batted_ball_location": "rf"},
{"half_inning": "Northwestern - Bottom of 5th", "play": "Erro flied out to cf.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Erro", "pa_result": "flied", "batted_ball_location": "cf"},
{"half_inning": "Northwestern - Bottom of 5th", "play": "Dickey out at second c to ss, caught stealing.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Dickey", "pa_result": ""},
{"half_inning": "OMAHA - Top of 6th", "play": "Thibodeau,C walked.", "offense_team": "OMAHA", "pitcher": "Bordignon", "name": "Thibodeau,C", "pa_result": "walked"},
{"half_inning": "OMAHA - Top of 6th", "play": "Hughes,H grounded into double play 2b to ss to 1b; Thibodeau,C out on the play.", "offense_team": "OMAHA", "pitcher": "Bordignon", "name": "Hughes,H", "pa_result": "grounded", "batted_ball_location": "2b"},
{"half_inning": "OMAHA - Top of 6th", "play": "Krumwiede,K singled to center field.", "offense_team": "OMAHA", "pitcher": "Bordignon", "name": "Krumwiede,K", "pa_result": "singled", "batted_ball_location": "center field"},
{"half_inning": "OMAHA - Top of 6th", "play": "Krumwiede,K advanced to second on a wild pitch.", "offense_team": "OMAHA", "pitcher": "Bordignon", "name": "Krumwiede,K", "pa_result": ""},
{"half_inning": "OMAHA - Top of 6th", "play": "Rogers,B walked.", "offense_team": "OMAHA", "pitcher": "Bordignon", "name": "Rogers,B", "pa_result": "walked"},
{"half_inning": "OMAHA - Top of 6th", "play": "Palensky,B struck out looking.", "offense_team": "OMAHA", "pitcher": "Bordignon", "name": "Palensky,B", "pa_result": "struck out looking"},
{"half_inning": "Northwestern - Bottom of 6th", "play": "Claeys singled to shortstop.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Claeys", "pa_result": "singled", "batted_ball_location": "p"},
{"half_inning": "Northwestern - Bottom of 6th", "play": "Lind grounded out to 2b; Claeys advanced to second.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Lind", "pa_result": "grounded", "batted_ball_location": "2b"},
{"half_inning": "Northwestern - Bottom of 6th", "play": "Dunn, D. singled to shortstop; Claeys advanced to third.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Dunn, D.", "pa_result": "singled", "batted_ball_location": "p"},
{"half_inning": "Northwestern - Bottom of 6th", "play": "Maxwell hit by pitch; Dunn, D. advanced to second.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Maxwell", "pa_result": "hit by pitch"},
{"half_inning": "Northwestern - Bottom of 6th", "play": "Moore,M to p for Suponchick,G.", "offense_team": "Northwestern", "pitcher": "Grant Suponchick", "name": "Moore,M", "pa_result": ""},
{"half_inning": "Northwestern - Bottom of 6th", "play": "Kelly walked, RBI; Maxwell advanced to second; Dunn, D. advanced to third; Claeys scored.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Kelly", "pa_result": "walked"},
{"half_inning": "Northwestern - Bottom of 6th", "play": "Bourbon grounded out to ss, RBI; Kelly advanced to second; Maxwell advanced to third; Dunn, D. scored.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Bourbon", "pa_result": "grounded", "batted_ball_location": "ss"},
{"half_inning": "Northwestern - Bottom of 6th", "play": "Dickey struck out swinging.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Dickey", "pa_result": "struck out swinging"},
{"half_inning": "OMAHA - Top of 7th", "play": "Gannon to p for Bordignon.", "offense_team": "OMAHA", "pitcher": "Bordignon", "name": "Gannon", "pa_result": ""},
{"half_inning": "OMAHA - Top of 7th", "play": "Caniglia,A walked.", "offense_team": "OMAHA", "pitcher": "Gannon", "name": "Caniglia,A", "pa_result": "walked"},
{"half_inning": "OMAHA - Top of 7th", "play": "Smejkal,P singled to right field; Caniglia,A advanced to second.", "offense_team": "OMAHA", "pitcher": "Gannon", "name": "Smejkal,P", "pa_result": "singled", "batted_ball_location": "right field"},
{"half_inning": "OMAHA - Top of 7th", "play": "Daugherty,T singled to third base, bunt; Smejkal,P advanced to second; Caniglia,A advanced to third.", "offense_team": "OMAHA", "pitcher": "Gannon", "name": "Daugherty,T", "pa_result": "singled", "batted_ball_location": "third base"},
{"half_inning": "OMAHA - Top of 7th", "play": "Gamm,M singled through the left side, RBI; Daugherty,T advanced to second; Smejkal,P advanced to third; Caniglia,A scored.", "offense_team": "OMAHA", "pitcher": "Gannon", "name": "Gamm,M", "pa_result": "singled", "batted_ball_location": "left side"},
{"half_inning": "OMAHA - Top of 7th", "play": "Thibodeau,C singled to center field, RBI; Gamm,M advanced to second; Daugherty,T advanced to third; Smejkal,P scored.", "offense_team": "OMAHA", "pitcher": "Gannon", "name": "Thibodeau,C", "pa_result": "singled", "batted_ball_location": "center field"},
{"half_inning": "OMAHA - Top of 7th", "play": "Lass to p for Gannon.", "offense_team": "OMAHA", "pitcher": "Gannon", "name": "Lass", "pa_result": ""},
{"half_inning": "OMAHA - Top of 7th", "play": "Hughes,H walked, RBI; Thibodeau,C advanced to second; Gamm,M advanced to third; Daugherty,T scored.", "offense_team": "OMAHA", "pitcher": "Lass", "name": "Hughes,H", "pa_result": "walked"},
{"half_inning": "OMAHA - Top of 7th", "play": "Krumwiede,K reached on a fielder's choice, RBI; Hughes,H out at second 3b to 2b; Thibodeau,C advanced to third; Gamm,M scored.", "offense_team": "OMAHA", "pitcher": "Lass", "name": "Krumwiede,K", "pa_result": "reached", "batted_ball_location": "3b"},
{"half_inning": "OMAHA - Top of 7th", "play": "Rogers,B walked; Krumwiede,K advanced to second.", "offense_team": "OMAHA", "pitcher": "Lass", "name": "Rogers,B", "pa_result": "walked"},
{"half_inning": "OMAHA - Top of 7th", "play": "Palensky,B singled to center field, RBI; Rogers,B advanced to second; Krumwiede,K advanced to third; Thibodeau,C scored.", "offense_team": "OMAHA", "pitcher": "Lass", "name": "Palensky,B", "pa_result": "singled", "batted_ball_location": "center field"},
{"half_inning": "OMAHA - Top of 7th", "play": "Caniglia,A singled to left field, RBI; Palensky,B advanced to second; Rogers,B advanced to third; Krumwiede,K scored.", "offense_team": "OMAHA", "pitcher": "Lass", "name": "Caniglia,A", "pa_result": "singled", "batted_ball_location": "left field"},
{"half_inning": "OMAHA - Top of 7th", "play": "Smejkal,P popped up to rf.", "offense_team": "OMAHA", "pitcher": "Lass", "name": "Smejkal,P", "pa_result": "popped", "batted_ball_location": "p"},
{"half_inning": "OMAHA - Top of 7th", "play": "Daugherty,T flied out to cf.", "offense_team": "OMAHA", "pitcher": "Lass", "name": "Daugherty,T", "pa_result": "flied", "batted_ball_location": "cf"},
{"half_inning": "Northwestern - Bottom of 7th", "play": "Dunn, J. struck out looking.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Dunn, J.", "pa_result": "struck out looking"},
{"half_inning": "Northwestern - Bottom of 7th", "play": "Erro walked.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Erro", "pa_result": "walked"},
{"half_inning": "Northwestern - Bottom of 7th", "play": "Claeys struck out looking.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Claeys", "pa_result": "struck out looking"},
{"half_inning": "Northwestern - Bottom of 7th", "play": "Erro stole second.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Erro", "pa_result": "stole"},
{"half_inning": "Northwestern - Bottom of 7th", "play": "Erro stole third.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Erro", "pa_result": "stole"},
{"half_inning": "Northwestern - Bottom of 7th", "play": "Lind struck out swinging.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Lind", "pa_result": "struck out swinging"},
{"half_inning": "OMAHA - Top of 8th", "play": "O'Laughlin to lf for Maxwell.", "offense_team": "OMAHA", "pitcher": "Lass", "name": "O'Laughlin", "pa_result": ""},
{"half_inning": "OMAHA - Top of 8th", "play": "Gamm,M lined out to cf.", "offense_team": "OMAHA", "pitcher": "Lass", "name": "Gamm,M", "pa_result": "lined", "batted_ball_location": "cf"},
{"half_inning": "OMAHA - Top of 8th", "play": "Thibodeau,C walked.", "offense_team": "OMAHA", "pitcher": "Lass", "name": "Thibodeau,C", "pa_result": "walked"},
{"half_inning": "OMAHA - Top of 8th", "play": "Hughes,H grounded into double play p to ss to 1b; Thibodeau,C out on the play.", "offense_team": "OMAHA", "pitcher": "Lass", "name": "Hughes,H", "pa_result": "grounded", "batted_ball_location": "p"},
{"half_inning": "Northwestern - Bottom of 8th", "play": "Dunn, D. tripled to right center.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Dunn, D.", "pa_result": "tripled", "batted_ball_location": "right center"},
{"half_inning": "Northwestern - Bottom of 8th", "play": "O'Laughlin singled through the right side, RBI; Dunn, D. scored.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "O'Laughlin", "pa_result": "singled", "batted_ball_location": "right side"},
{"half_inning": "Northwestern - Bottom of 8th", "play": "Kelly lined out to rf.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Kelly", "pa_result": "lined", "batted_ball_location": "rf"},
{"half_inning": "Northwestern - Bottom of 8th", "play": "Bourbon struck out swinging.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Bourbon", "pa_result": "struck out swinging"},
{"half_inning": "Northwestern - Bottom of 8th", "play": "Dickey doubled down the rf line; O'Laughlin advanced to third.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Dickey", "pa_result": "doubled", "batted_ball_location": "rf"},
{"half_inning": "Northwestern - Bottom of 8th", "play": "Dunn, J. singled to left field, 2 RBI; Dickey scored; O'Laughlin scored.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Dunn, J.", "pa_result": "singled", "batted_ball_location": "left field"},
{"half_inning": "Northwestern - Bottom of 8th", "play": "Dunn, J. stole second.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Dunn, J.", "pa_result": "stole"},
{"half_inning": "Northwestern - Bottom of 8th", "play": "Erro reached on a throwing error by ss; Dunn, J. advanced to third.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Erro", "pa_result": "reached", "batted_ball_location": "ss"},
{"half_inning": "Northwestern - Bottom of 8th", "play": "Jones to c for Claeys.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Jones", "pa_result": ""},
{"half_inning": "Northwestern - Bottom of 8th", "play": "Dunn, J. scored on the throw, unearned.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Dunn, J.", "pa_result": ""},
{"half_inning": "Northwestern - Bottom of 8th", "play": "Jones walked; Erro advanced to second.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Jones", "pa_result": "walked"},
{"half_inning": "Northwestern - Bottom of 8th", "play": "Lind struck out swinging.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Lind", "pa_result": "struck out swinging"},
{"half_inning": "OMAHA - Top of 9th", "play": "Krumwiede,K singled to center field.", "offense_team": "OMAHA", "pitcher": "Lass", "name": "Krumwiede,K", "pa_result": "singled", "batted_ball_location": "center field"},
{"half_inning": "OMAHA - Top of 9th", "play": "Rogers,B grounded into double play 2b to ss to 1b; Krumwiede,K out on the play.", "offense_team": "OMAHA", "pitcher": "Lass", "name": "Rogers,B", "pa_result": "grounded", "batted_ball_location": "2b"},
{"half_inning": "OMAHA - Top of 9th", "play": "Palensky,B flied out to rf.", "offense_team": "OMAHA", "pitcher": "Lass", "name": "Palensky,B", "pa_result": "flied", "batted_ball_location": "rf"},
{"half_inning": "Northwestern - Bottom of 9th", "play": "Steier,T to p for Moore,M.", "offense_team": "Northwestern", "pitcher": "Moore,M", "name": "Steier,T", "pa_result": ""},
{"half_inning": "Northwestern - Bottom of 9th", "play": "Dunn, D. flied out to cf.", "offense_team": "Northwestern", "pitcher": "Steier,T", "name": "Dunn, D.", "pa_result": "flied", "batted_ball_location": "cf"},
{"half_inning": "Northwestern - Bottom of 9th", "play": "O'Laughlin grounded out to ss.", "offense_team": "Northwestern", "pitcher": "Steier,T", "name": "O'Laughlin", "pa_result": "grounded", "batted_ball_location": "ss"},
{"half_inning": "Northwestern - Bottom of 9th", "play": "Kelly grounded out to 2b.", "offense_team": "Northwestern", "pitcher": "Steier,T", "name": "Kelly", "pa_result": "grounded", "batted_ball_location": "2b"}
])

,batted_ball_location,half_inning,name,offense_team,pa_result,pitcher,play
0,center field,OMAHA - Top of 1st,"Thibodeau,C",OMAHA,singled,"Christie, Hank","Thibodeau,C singled to center field."
1,NaN,OMAHA - Top of 1st,"Thibodeau,C",OMAHA,stole,"Christie, Hank","Thibodeau,C stole second."
2,NaN,OMAHA - Top of 1st,"Hughes,H",OMAHA,struck out looking,"Christie, Hank","Hughes,H struck out looking."
3,ss,OMAHA - Top of 1st,"Krumwiede,K",OMAHA,grounded,"Christie, Hank","Krumwiede,K grounded out to ss."
4,p,OMAHA - Top of 1st,"Rogers,B",OMAHA,grounded,"Christie, Hank","Rogers,B grounded out to p."
5,p,Northwestern - Bottom of 1st,Dickey,Northwestern,grounded,Grant Suponchick,Dickey grounded out to p.
6,third base,Northwestern - Bottom of 1st,"Dunn, J.",Northwestern,singled,Grant Suponchick,"Dunn, J. singled to third base."
7,right center,Northwestern - Bottom of 1st,Erro,Northwestern,singled,Grant Suponchick,"Erro singled to right center; Dunn, J. advance..."
8,NaN,Northwestern - Bottom of 1st,Erro,Northwestern,stole,Grant Suponchick,"Erro stole second; Dunn, J. stole home."
9,NaN,Northwestern - Bottom of 1st,Claeys,Northwestern,struck out swinging,Grant Suponchick,"Claeys struck out swinging, out at first c to ..."


In [ ]:
def teamid_lookup(team):
    team = team.lower()
    r = requests.get()
    
def playerid_lookup(last, first=None, teamid, year=2018):
    # force input strings to lowercase
    last = last.lower()
    if first:
        first = first.lower()
    if first is None:
        results = table.loc[table['name_last']==last]
    else:
        results = table.loc[(table['name_last']==last) & (table['name_first']==first)]
    #results[['key_mlbam', 'key_fangraphs', 'mlb_played_first', 'mlb_played_last']] = results[['key_mlbam', 'key_fangraphs', 'mlb_played_first', 'mlb_played_last']].astype(int) # originally returned as floats which is wrong
    results = results.reset_index().drop('index', 1)
    return results

In [43]:
import json

r = requests.get('http://thebaseballcube.com/college/teams.asp?L=1')
soup = BeautifulSoup(r.text, 'html.parser')
dataRows = soup.find_all(class_='dataRow')
team_ids = {}
pattern = re.compile(r'(?<=T=)\w+')
for row in dataRows:
    team = row.a.get_text()
    team_id = re.search(pattern, row.a['href'])
    team_ids[team] = int(team_id.group(0))

l = []
for team in team_ids:
    l.append({
        'id': team_ids[team],
        'name': team
    })
json.dumps(l)

'[{"id": 20366, "name": "Abilene Christian"}, {"id": 20770, "name": "Alabama A&M;"}, {"id": 20368, "name": "Alabama State"}, {"id": 20369, "name": "Alcorn State"}, {"id": 20370, "name": "Appalachian State"}, {"id": 20021, "name": "Arizona State"}, {"id": 20371, "name": "Arkansas State"}, {"id": 20071, "name": "Auburn"}, {"id": 20373, "name": "Austin Peay"}, {"id": 20103, "name": "Ball State"}, {"id": 20287, "name": "Baylor"}, {"id": 20828, "name": "Belmont"}, {"id": 20374, "name": "Bethune-Cookman"}, {"id": 20886, "name": "Binghamton"}, {"id": 20291, "name": "Boston College"}, {"id": 20280, "name": "Bowling Green State"}, {"id": 20242, "name": "Bradley"}, {"id": 20376, "name": "Brigham Young"}, {"id": 20361, "name": "Brown"}, {"id": 21866, "name": "Bryant University"}, {"id": 20518, "name": "Bucknell"}, {"id": 20705, "name": "Butler"}, {"id": 20740, "name": "Cal Poly"}, {"id": 21135, "name": "Cal State Bakersfield"}, {"id": 20100, "name": "Cal State Fullerton"}, {"id": 20106, "name": "

In [34]:
team_ids = {'abilene christian': 20366, 'alabama a&m;': 20770, 'alabama state': 20368, 'alcorn state': 20369, 'appalachian state': 20370, 'arizona state': 20021, 'arkansas state': 20371, 'auburn': 20071, 'austin peay': 20373, 'ball state': 20103, 'baylor': 20287, 'belmont': 20828, 'bethune-cookman': 20374, 'binghamton': 20886, 'boston college': 20291, 'bowling green state': 20280, 'bradley': 20242, 'brigham young': 20376, 'brown': 20361, 'bryant university': 21866, 'bucknell': 20518, 'butler': 20705, 'cal poly': 20740, 'cal state bakersfield': 21135, 'cal state fullerton': 20100, 'cal state northridge': 20106, 'campbell': 20880, 'canisius college': 21093, 'central ct state': 20108, 'central michigan': 20270, 'charleston southern': 20522, 'chicago state': 21388, 'clemson': 20089, 'coastal carolina': 20319, 'college of charleston': 21014, 'holy cross': 20800, 'william and mary': 20486, 'columbia': 21931, 'coppin state college': 21094, 'cornell': 20295, 'creighton': 20115, 'dallas baptist': 20117, 'dartmouth': 20038, 'davidson college': 20529, 'delaware state': 20383, 'duke': 20118, 'east carolina': 20384, 'east tennessee state': 20385, 'eastern illinois': 20174, 'eastern kentucky': 20175, 'eastern michigan': 20447, 'elon': 20537, 'fairfield': 20835, 'fairleigh dickinson': 20982, 'florida a&m;': 20079, 'florida atlantic': 20543, 'florida gulf coast': 21988, 'florida intl': 20041, 'florida state': 20022, 'fordham': 20122, 'fresno state': 20033, 'furman': 20292, 'gardner-webb': 21025, 'george mason': 20123, 'george washington': 20070, 'georgetown': 20489, 'georgia southern': 20090, 'georgia state': 21026, 'georgia tech': 20124, 'gonzaga': 20239, 'grambling state': 20083, 'grand canyon': 20125, 'harvard': 20257, 'high point': 21090, 'hofstra': 20393, 'houston baptist': 20952, 'illinois state': 20129, 'indiana state': 20256, 'indiana': 20452, 'indiana u.-purdue u.-fw': 22025, 'iona college': 20565, 'jackson state': 20396, 'jacksonville state': 20131, 'jacksonville': 20566, 'james madison': 20132, 'kansas state': 20398, 'kennesaw state': 20134, 'kent state': 20135, 'la salle': 21089, 'lafayette college': 21098, 'lamar': 20040, 'lehigh': 20401, 'liberty': 20403, 'lipscomb': 20846, 'long beach state': 20010, 'long island-brooklyn': 20779, 'longwood': 20276, 'lsu': 20004, 'louisiana tech': 20140, 'loyola marymount': 20065, 'manhattan college': 20790, 'marist': 20577, 'marshall': 20240, 'mcneese state': 20142, 'mercer': 20246, 'miami': 20182, 'michigan state': 20145, 'middle tenn state': 20204, 'mississippi state': 20147, 'miss. valley st': 20408, 'missouri state': 20219, 'monmouth': 22104, 'morehead state': 20588, 'mount saint mary': 21052, 'murray state': 20248, 'navy': 20410, 'new jersey tech': 22122, 'new mexico state': 20491, 'new york tech': 20069, 'niagara': 21054, 'nicholls state': 20216, 'norfolk state': 20411, 'nc a&t; state': 20412, 'north carolina central': 22353, 'nc state': 20235, 'north dakota state': 20413, 'northeastern': 20328, 'northern illinois': 20463, 'northern kentucky': 20860, 'northwestern state': 20317, 'northwestern': 20037, 'oakland': 21632, 'ohio state': 20152, 'ohio': 20499, 'oklahoma state': 20093, 'old dominion': 20153, 'oral roberts': 20024, 'oregon state': 20272, 'pacific': 20241, 'penn state': 20348, 'pepperdine': 20074, 'prairie view a&m;': 20771, 'presbyterian college': 21656, 'princeton': 20326, 'purdue': 20417, 'quinnipiac': 20288, 'radford': 20606, 'rice': 20032, 'rider': 20739, 'rutgers': 20097, 'sacramento state': 20419, 'sacred heart': 20610, "saint joseph's": 20218, 'saint louis': 20500, "saint peter's": 21092, 'sam houston state': 20258, 'samford': 20053, 'san diego state': 20001, 'san jose state': 20161, 'santa clara': 20296, 'savannah state': 20421, 'seattle': 21265, 'seton hall': 20165, 'siena college': 20615, 'south dakota state': 20425, 'se missouri state': 20245, 'se louisiana': 20294, 'southern illinois': 20005, 'siu-edwardsville': 20190, 'southern': 20428, 'st. bonaventure': 20734, "st. john's": 20036, "st. mary's (ca)": 20217, 'stanford': 20039, 'stephen f. austin': 20429, 'stetson': 20307, 'stony brook': 20631, 'tennessee tech': 20249, 'texas a&m;': 20023, 'texas a&m-corpus; christi': 22240, 'texas christian': 20433, 'texas southern': 20434, 'texas state': 20016, 'texas tech': 20169, 'the citadel': 20334, 'towson': 20435, 'troy': 20336, 'tulane': 20173, 'ucla': 20054, 'air force': 20367, 'suny - albany': 20881, 'alabama': 20048, 'alabama-birmingham': 20439, 'arizona': 20026, 'arkansas': 20344, 'ark-little rock': 20492, 'arkansas-pine bluff': 20440, 'california': 20008, 'uc davis': 20442, 'uc irvine': 20029, 'uc riverside': 20232, 'uc santa barbara': 20304, 'central arkansas': 20494, 'central florida': 20443, 'cincinnati': 20357, 'connecticut': 20223, 'dayton': 20896, 'delaware': 20078, 'evansville': 20176, 'florida': 20177, 'georgia': 20350, 'hartford': 20045, 'hawaii': 20450, 'houston': 20254, 'illinois': 20178, 'illinois-chicago': 20891, 'iowa': 20179, 'kansas': 20454, 'kentucky': 20017, 'louisiana-lafayette': 20648, 'louisiana-monroe': 20456, 'louisville': 20457, 'maine': 20180, 'maryland': 20213, 'umd-baltimore cty': 20297, 'maryland-eastern shore': 21095, 'massachusetts': 20181, 'massachusetts-lowell': 20346, 'memphis': 20018, 'michigan': 20183, 'minnesota': 20011, 'mississippi': 20085, 'missouri': 20458, 'nebraska': 20066, 'nebraska at omaha': 20460, 'nevada': 20255, 'new mexico': 20266, 'new orleans': 20498, 'north carolina': 20006, 'unc wilmington': 20652, 'unc asheville': 20861, 'unc charlotte': 20502, 'unc-greensboro': 20215, 'north florida': 20342, 'northern colorado': 20462, 'notre dame': 20151, 'oklahoma': 20214, 'oregon': 20465, 'pennsylvania': 20185, 'pittsburgh': 20358, 'portland': 20186, 'rhode island': 20466, 'richmond': 20092, 'san diego': 20187, 'san francisco': 20325, 'south alabama': 20009, 'south carolina': 20091, 'usc-upstate': 21767, 'south florida': 20353, 'southern miss': 20049, 'tennessee': 20015, 'tennessee-martin': 22297, 'texas': 20193, 'texas-arlington': 20659, 'texas-san antonio': 21083, 'incarnate word': 20904, 'toledo': 20262, 'utah': 20472, 'virginia': 20194, 'washington': 20318, 'uw milwaukee': 20911, 'unlv': 20196, 'army': 21097, 'usc': 20064, 'texas-pan american': 20809, 'utah valley u': 20966, 'valparaiso': 20503, 'vanderbilt': 20231, 'villanova': 20198, 'va. commonwealth': 20072, 'va military inst.': 20665, 'virginia tech': 20293, 'wagner': 21096, 'wake forest': 20094, 'washington state': 20057, 'west virginia': 20195, 'western carolina': 20485, 'western illinois': 20474, 'western kentucky': 20670, 'western michigan': 20277, 'wichita state': 20031, 'winthrop': 20701, 'wofford': 21091, 'wright state': 20030, 'xavier': 20506, 'yale': 20314, 'youngstown state': 20488}